# **Exploratory Data Analysis (EDA)**

This notebook aims to gain insights into the patterns, trends, and characteristics of our dataset. Through data exploration and visualization, our objective is to understand the underlying patterns, detect anomalies, and better understand energy demand behavior in the UK. We will utilize visualizations, statistical analysis, and decomposition techniques to delve into the dataset and uncover valuable insights. This exploration will lay the groundwork for subsequent analysis and modeling, empowering us to make informed decisions and accurate predictions.

## **Table of Contents**

1. Imports and Data Load
2. General Visualization
4. Data Preview
    - National Demand (target variable)
    - Desagregated Energy Demand/Consumption
    - Wind and Solar energy
    - Import and export flows
5. Data Decomposition 
    - Seasonal Plot
    - Trend Seasonal Decomposition
6. Data Saving

### **1. Imports and Data Load**

In [ ]:
#Import all relevant libraries
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from statsmodels.graphics.tsaplots import month_plot

from statsmodels.tsa.seasonal import seasonal_decompose

from datetime import datetime, timedelta

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#read csv and make firts column the index 
df = pd.read_csv('../data/historic_demand_2009_2023_cleaned.csv', index_col=0)
df_monthly = pd.read_csv('../data/monthly_historic_demand_2009_2023_cleaned.csv', index_col=0)

In [ ]:
#Check the read was ok
df.head(3).T

In [ ]:
#Check index type, when save to a csv the index type is not stored
print(type(df.index))
print(type(df_monthly.index))

In [ ]:
#Make the index datetime 
df = df.set_index(pd.to_datetime(df.index))
df_monthly = df_monthly.set_index(pd.to_datetime(df_monthly.index))

#Sanity check of index characteristics 
print(f"Index data type of df is: {type(df.index)}")
print(f"Index data type of df_monthly is: {type(df_monthly.index)}")
print(f"Index in increasing order: {df.index.is_monotonic_increasing}, {df_monthly.index.is_monotonic_increasing}")
print(f"Index with unique values: {df.index.is_unique}, {df_monthly.index.is_unique}")

### **2. General Visualization**

In thise section we are going to visualize several columns to understand how data is distributed, identify patterns and look for missing or unusual values. 

In [ ]:
#Create list for visualization purposes
df_list = ['df', 'df_daily', 'df_monthly']

#### **National Demand Analysis** (Target variable)



In [ ]:
#National Demand (nd) over time
fig = make_subplots(rows=3, cols=1, subplot_titles=df_list)

for i, df_name in enumerate(df_list):
    fig.add_trace(go.Scatter(x=eval(df_name).index, y=eval(df_name)['nd'], mode='lines'), row=i+1, col=1)
    fig.update_yaxes(title_text='MW', row=i+1, col=1)

fig.update_layout(title='<b>National Demand</b>',  title_x=0.5,  height=800, width=1000, xaxis_title='Date', showlegend=False)

fig.show()

National demand exhibits a slight downward trend and clear seasonality. However, the variability or dispersion of the data appears to be lower in the later years.

In the df_monthly plot, we can observe that in July 2010 and 2012, the data displays sharp downward and upward curves. These fluctuations become smoother over time as the years progress.

#### **Desagregated energy demand**

The plot below displays the aggregate energy demand in the UK. The purple line represents the total energy demand of the system (TSD), the red line represents the National Demand (our target variable), and the green line represents the demand in England and Wales. These variables exhibit a high degree of collinearity as they contain overlapping information. However, it is important to note that there are subtle differences between National Demand and Transmission System Demand (TSD). TSD includes the energy consumed by the system for operational purposes, accounting for the slight variations between the purple and red plot lines.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['tsd'], name='Transmitting System Demand'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['nd'], name='National Demand'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['england_wales_demand'], name='England Wales Demand'))

fig.update_layout(title='<b>Desagregated Electricity Demand</b>', title_x=0.5, xaxis_title='Time', yaxis_title='Demand (MW)')
fig.update_xaxes(rangeslider_visible=True)
fig.show()


#### **Wind and Solar**

In [ ]:
#Plot solar and wind generation and capacity
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['embedded_wind_generation'], name='Wind Generation'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['embedded_wind_capacity'], name='Wind Capacity'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['embedded_solar_generation'], name='Solar Generation'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['embedded_solar_capacity'], name='Solar Capacity'))
fig.update_layout(title='<b>Wind and Solar generation and capacity</b>', title_x=0.5, xaxis_title='Time', yaxis_title='MW')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

The plot above displays four columns related to wind and solar energy sources. We can observe a gradual increase in generation for both sources, which plateaus after 2016. In terms of capacity, solar energy experienced substantial and rapid growth in 2015, followed by stabilization around 2018. Interestingly, it is notable that the generation does not proportionally increase despite the growth in capacity. Furthermore, it is worth mentioning that the highs and lows of solar and wind generation appear to be asynchronous. This means that when one source is generating at a high level, the other source tends to have a lower generation, and vice versa.

#### **Import and export flows**

From the data description we know that `ifa_flow`, `ifa2_flow`, `britned_flow`, `moyle_flow`,`east_west_flow` and `nemo_flow` all are import/export columns so we will merge them in a single column. 

In [ ]:
df_monthly['import_export'] = (
    df_monthly['ifa_flow'] +
    df_monthly['ifa2_flow'] +
    df_monthly['britned_flow'] +
    df_monthly['moyle_flow'] +
    df_monthly['east_west_flow'] +
    df_monthly['nemo_flow']
)

In [ ]:
#Sanity check
total_import_export_monthly = (
    df_monthly['ifa_flow'] +
    df_monthly['ifa2_flow'] +
    df_monthly['britned_flow'] +
    df_monthly['moyle_flow'] +
    df_monthly['east_west_flow'] +
    df_monthly['nemo_flow']
)

total_import_export_monthly_sum = total_import_export_monthly.sum() #Add values 
print(total_import_export_monthly_sum - df_monthly['import_export'].sum()) 

In [ ]:
#Check results
fig, axes = plt.subplots(2, 1, figsize=(10, 5))

axes[0].plot(df_daily['import_export'])
axes[0].set_title('Daily Import/Export')
axes[1].plot(df_monthly['import_export'])
axes[1].set_title('Monthly Import/Export')
plt.tight_layout()

plt.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['ifa_flow'], name='IFA'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['ifa2_flow'], name='IFA2'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['britned_flow'], name='BRITNED'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['moyle_flow'], name='MOYLE'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['east_west_flow'], name='EAST-WEST'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['nemo_flow'], name='NEMO'))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly['import_export'], name='Consolidated Import-Export'))
fig.update_layout(title='<b>Import and Export Flows</b>', title_x=0.5, xaxis_title='Time', yaxis_title='MW')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
#Drop added columns 

df_monthly.drop(columns=['ifa_flow', 'ifa2_flow', 'britned_flow', 'moyle_flow','east_west_flow', 'nemo_flow'], inplace=True)
df_monthly.columns

### **4. Data Decomposition**
To understand energy consumption patters we are going to plot a breakdown of National Demand by hour, weeks and months. 

In [ ]:
#Create features to plot
df['hour'] = df.index.hour
df['day_week'] = df.index.dayofweek
df['month'] = df.index.month

In [ ]:
#plot energy consumption by hour 
fig, ax = plt.subplots(figsize=(10, 4))
sns.boxplot(data=df, x='hour', y='nd')
sns.despine()
ax.set_title("MW by hour")
plt.show()

Energy consumtion during the first hours of the day is low, raching it lowest point at 4 AM, afterwars it rises from 5 to 9 AM when it stabilizes until 3 PM when it surges to reach a high peak at 6 PM and finally steadily declines until midgnight. 

In [ ]:
#plot energy consumption by week
fig, ax = plt.subplots(figsize=(10, 4))
sns.boxplot(data=df, x='day_week', y='nd')
sns.despine()
ax.set_title("MW by day of Week")
ax.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun'])
plt.show()


Energy consumption is fairly similar from monday to friday, the weekend present a lower energy consumtion. 

In [ ]:
#plot energy consumption by year
fig, ax = plt.subplots(figsize=(10,4))
sns.boxplot(data=df, x='month', y='nd')
sns.despine()
ax.set_title("MW by month")
plt.show()

Energy consumption through the year has a clear seasonal pattern with low consumption in the warmer months and high consumption in the cold winter months. 

### **Seasonal Plot**

Another important aspect to analyze in time series data is the presence of seasonality and the components of the series. In the following plots, we will visualize the monthly consumption and the decomposition of the National Demand column, providing valuable insights into the data.

In [ ]:
plt.figure(figsize=(15, 5))

month_plot(df_monthly["nd"], ax=plt.gca())

plt.title("Seasonal National Demand per Month")
sns.despine()
plt.show()

The monthly trend of the energy consumption has a U shape with its highest points on January and December and the lowest point of energy consumption in the summer month of June. 

Its intereseting to notice the upward trend of or the later years in April, May and June. 

### **Trend Season Decomposition**

We will employ an additive method for the seasonal decomposition.

The seasonal decomposition can be described as follows:
$$
yt = St + Tt + Rt
$$
In the equation above, yt represents the observation at a certain period t (`nd` at period t). St represents the seasonal component of the series, which in our case is an annual pattern. Tt represents the trend of the series, which in the case of energy is ascending. Rt represents the random component of the data, which we expect to exhibit no clear pattern, seasonality, or trend. The elements being decomposed are added together because they do not have a multiplicative effect on each other; their sum constitutes the data.

In [ ]:
# decompose the time series using df_monthly
decomposition_nd = seasonal_decompose(df_monthly['nd'], model='additive')

In [ ]:
#Save results
df_monthly["trend_nd"] = decomposition_nd.trend
df_monthly["seasonal_nd"] = decomposition_nd.seasonal
df_monthly["residual_nd"] = decomposition_nd.resid

In [ ]:
#Plot resutls
cols = ["trend_nd", "seasonal_nd", "residual_nd"]

fig = make_subplots(rows=3, cols=1, subplot_titles=cols)

for i, col in enumerate(cols):
    fig.add_trace(
        go.Scatter(x=df_monthly.index, y=df_monthly[col]),
        row=i+1,
        col=1
    )

fig.update_layout(title="<b>National Demand Decomposition</b>", title_x=0.5, height=800, width=1050, showlegend=False)
fig.show()


From the plot above we can conclude the following:

- **Trend:** The National Demand for energy shows a decreasing trend with occasional small upward bumps. The decline in energy consumption has been steady since 2013. 

- **Seasonality:**  The National Demand column exhibits a clear seasonal pattern that aligns with weather seasons. There is high demand during the cold months of winter and low energy consumption during summer, with the lowest point occurring in June. March shows a slight increase from the downward slope of the seasonal pattern, which could be attributed to Easter.

- **Residuals:** The residual component appears to be random, with generally low variation, except for a few steep high and low peaks.

### **5. Data Saving**

In [ ]:
df.to_csv('../data/historic_demand_2009_2023_prepro.csv', index=True)
df_monthly.to_csv('../data/monthly_historic_demand_2009_2023_prepro.csv', index=True)